In [3]:
from pysnmp.entity.rfc3413.oneliner import cmdgen

cmd = cmdgen.CommandGenerator()

In [11]:
errorIndication, errorStatus, errorIndex, varBinds = cmd.getCmd(
    # v1
    # cmdgen.CommunityData('public', 0),
    # v2
    # cmdgen.CommunityData('public'),
    # v3
    cmdgen.UsmUserData(
        'demo', 'my_new_password', 'my_new_password', 
        cmdgen.usmHMACMD5AuthProtocol,
        cmdgen.usmDESPrivProtocol
    ),
    cmdgen.UdpTransportTarget(('ss.laisky.com', 161)),
    cmdgen.MibVariable('1.3.6.1.2.1.1.1.0')
)

print(
    'errorIndication: {}\n'
    'errorStatus: {}\n'
    'errorIndex: {}\n'
    'varBinds: {}'
    .format(errorIndication, errorStatus, errorIndex, varBinds)
)

errorIndication: None
errorStatus: 0
errorIndex: 0
varBinds: [ObjectType(ObjectIdentity(ObjectName('1.3.6.1.2.1.1.1.0')), DisplayString(b'Linux li744-109 4.1.5-x86_64-linode61 #7 SMP Mon Aug 24 13:46:31 EDT 2015 x86_64', subtypeSpec=ConstraintsIntersection(ConstraintsIntersection(ConstraintsIntersection(ConstraintsIntersection(), ValueSizeConstraint(0, 65535)), ValueSizeConstraint(0, 255)), ValueSizeConstraint(0, 255))))]


chexiang
===

- system = 1.3.6.1.2.1.1
- sysName = 1.3.6.1.2.1.1.5
- ifEntry = 1.3.6.1.2.1.2.2.1
- ifDescr = 1.3.6.1.2.1.2.2.1.2
- ifSpeed = 1.3.6.1.2.1.2.2.1.5

In [15]:
import string
from pysnmp.smi import builder, view, error
from pysnmp.entity.rfc3413.oneliner import cmdgen
from collections import namedtuple as NT

cmdGen = cmdgen.CommandGenerator()
mibBuilder = builder.MibBuilder()
mibBuilder.loadModules(
    'RFC-1213', 'RFC-3413'
    )
mibView = view.MibViewController(mibBuilder)
errorIndication, errorStatus, errorIndex, \
                 varBindTable = cmdGen.nextCmd(
    # SNMP v2
    cmdgen.CommunityData('huawei@123'),
    # Transport
    cmdgen.UdpTransportTarget(('10.32.33.10', 161)),
    cmdgen.MibVariable(*('IF-MIB', 'ifOutOctets')),
#     cmdgen.MibVariable('IF-MIB', 'ifDescr'),
)


print(errorIndication)
print(errorStatus)
print(errorIndex)
print('*' * 50)

    
for row in varBindTable:
    for oid, val in row:
#         print(repr(oid))
        print(oid.getMibSymbol()[-1][0], val.prettyPrint())

None
0
0
**************************************************
1 0
2 0
3 27370
5 677095921
6 662419672
7 661712657
8 303363872
9 659786839
10 1013002893
11 645175091
12 675785726
13 643100720
14 1475313986
15 1004680347
16 1010004827
17 282956583
18 283184569
19 296412774
20 2663611664
21 3647534609
22 1967238514
23 4285917384
24 2743781138
25 969838713
26 474340758
27 3818671621
28 97924068
29 1840288398
30 1424314283
31 1549512079
32 1500687573
33 1894781206
34 690128012
35 1379100984
36 1383782654
37 1575579371
38 1515670661
39 1558856185
40 3374178554
41 3295492557
42 2316269418
43 3991249265
44 1480662140
45 108262007
46 1398462092
47 1883561856
48 2908494113
49 487581795
50 2303653936
51 78080134
52 3862043673
53 2909986333
54 0
55 0
56 0
57 0
58 0
59 0
60 2909986333


In [21]:
a = val.prettyPrint()
a

'2909986333'

In [30]:
d = oid.getMibSymbol()
d

('IF-MIB', 'ifInOctets', (InterfaceIndex(60),))

In [3]:
o = cmdgen.MibVariable('IF-MIB', 'ifInOctets', 1)
o.loadMibs()

ObjectIdentity('IF-MIB', 'ifInOctets', 1)